In [28]:
import pandas as pd
import networkx as nx
from collections import deque
import geopandas as gpd

In [29]:
parquet_file = "/Users/rachel1/Downloads/model-metadata-table.parquet"
print("have parquet file")

# Read the Parquet file into a DataFrame
meta_data = pd.read_parquet(parquet_file, engine="pyarrow")

have parquet file


In [30]:
meta_data

,LINKNO,DSLINKNO,strmOrder,USContArea,DSContArea,TDXHydroRegion,VPUCode,TopologicalOrder,LengthGeodesicMeters,TerminalLink,musk_k,musk_x
0,110007873,110009186,2,1.083060e+07,4.209271e+07,1020000010,102,177181,13384.058990,110013122,53536,0.25
1,110010498,110011810,2,1.096513e+07,2.296918e+07,1020000010,102,177182,2201.649965,110013122,8807,0.25
2,110005251,110019682,2,1.111977e+07,1.703163e+07,1020000010,102,177183,1972.054239,110030180,7888,0.25
3,110018371,110014434,2,1.076590e+07,3.751075e+07,1020000010,102,177184,5305.002476,110030180,21220,0.25
4,110009187,110011811,2,1.321197e+07,2.302526e+07,1020000010,102,177185,3587.784457,110030180,14351,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...
6254099,820247408,820279661,9,1.666337e+12,1.666337e+12,8020008900,804,235109,3235.850704,820224554,3050,0.25
6254100,820279661,820281005,9,1.666343e+12,1.666378e+12,8020008900,804,235111,1513.076667,820224554,1426,0.25
6254101,820281005,820247403,9,1.666545e+12,1.666550e+12,8020008900,804,235112,3384.104153,820224554,3190,0.25
6254102,820247403,820224554,9,1.666565e+12,1.666577e+12,8020008900,804,235114,2834.209687,820224554,2671,0.25


In [31]:
df = meta_data[meta_data['VPUCode'].astype(str).str.startswith('7')]

df

,LINKNO,DSLINKNO,strmOrder,USContArea,DSContArea,TDXHydroRegion,VPUCode,TopologicalOrder,LengthGeodesicMeters,TerminalLink,musk_k,musk_x
5243964,710009923,710017859,2,9.757764e+06,1.774100e+07,7020000010,702,167482,983.215740,710194483,3933,0.25
5243965,710011907,710021828,2,9.961045e+06,2.049310e+07,7020000010,702,167483,1300.083005,710194483,5200,0.25
5243966,710013891,710015875,2,1.318300e+07,1.837337e+07,7020000010,702,167484,1985.630403,710194483,7943,0.25
5243967,710005956,710015876,2,1.003304e+07,1.718574e+07,7020000010,702,167485,3632.552079,710194483,14530,0.25
5243968,710007941,710021830,2,1.181393e+07,2.802320e+07,7020000010,702,167486,3876.816305,710194483,15507,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...
6103714,780050562,780050946,6,8.466326e+09,8.471870e+09,7020065090,718,24443,5724.453811,780057855,13129,0.25
6103715,780050946,780048641,6,8.481928e+09,8.482809e+09,7020065090,718,24444,2943.494318,780057855,6751,0.25
6103716,780048641,780049025,6,8.488289e+09,8.489888e+09,7020065090,718,24445,4805.185059,780057855,11021,0.25
6103717,780049025,780057855,6,8.505444e+09,8.514673e+09,7020065090,718,24446,34094.938106,780057855,78199,0.25


In [32]:

# Example: Assuming df has columns 'LINKNO' and 'DSLINKNO'
G = nx.DiGraph()

# Add edges from LINKNO to DSLINKNO
edges = df[['LINKNO', 'DSLINKNO']].dropna().itertuples(index=False, name=None)
G.add_edges_from(edges)

In [33]:
def count_upstream_links(G, linkno):
    # Get all nodes that can reach the given linkno (predecessors)
    upstream_links = nx.ancestors(G, linkno)
    return len(upstream_links)

In [34]:
linkno = 710450998 # Replace with the desired LINKNO
num_upstream = count_upstream_links(G, linkno)
print(f"Number of upstream links for {linkno}: {num_upstream}")

Number of upstream links for 710450998: 40190


In [35]:
# Assuming you have a DataFrame `df` with 'LINKNO' and 'DSLINKNO'
# Filter out rows where 'LINKNO' or 'DSLINKNO' is -1
filtered_df = df[(df['LINKNO'] != -1) & (df['DSLINKNO'] != -1)]

# Create a directed graph from the filtered dataframe
G_filtered = nx.from_pandas_edgelist(filtered_df, 'LINKNO', 'DSLINKNO', create_using=nx.DiGraph())

# Now G_filtered is your graph without -1 values


In [36]:
def find_unique_nodes_with_50000_upstreams(G_filtered, threshold=6000):
    result = []
    covered_links = set()  # Store already processed upstream nodes

    # Identify downstream-most nodes (nodes with no successors)
    downstream_nodes = [node for node in G_filtered.nodes if G_filtered.out_degree(node) == 0]

    for node in downstream_nodes:
        if node in covered_links:
            continue  # Skip if this node's upstreams are already counted

        upstream_count = 0
        visited = set()
        queue = deque([node])

        while queue:
            current = queue.popleft()
            if current in visited:
                continue
            visited.add(current)

            predecessors = list(G.predecessors(current))
            upstream_count += len(predecessors)

            if upstream_count > threshold:  # Stop early if we exceed the threshold
                result.append(node)  # Keep nodes exceeding threshold
                covered_links.update(visited)  # Mark all upstream nodes as covered
                break

            queue.extend(predecessors)

    return result

# Run the function
filtered_linknos = find_unique_nodes_with_50000_upstreams(G_filtered, threshold=6000)

print(filtered_linknos)


[710235268, 710441083, 720195572, 720164197, 730140925, 730231962, 740329561, 740603080, 750252614, 760049883, 760096444, 770490167, 770390434, 770448577]


In [16]:
nexus_points = gpd.read_file("/Users/rachel1/Downloads/global_nexus.gpkg")

In [24]:
while True:
    ds_counts = df["DSLINKNO"].value_counts()

    new_linknos = []
    changes_made = False

    for linkno in filtered_linknos:
        if ds_counts.get(linkno, 0) == 2:
            new_linknos.append(linkno)
        else:
            replacement = df.loc[df["DSLINKNO"] == linkno, "LINKNO"].values
            print(replacement)
            if len(replacement) > 0:
                new_value = replacement[0]
                if new_value != linkno:  # Ensure there is an actual change
                    changes_made = True
                new_linknos.append(new_value)
            else:
                new_linknos.append(linkno)

    if not changes_made:  # Stop when no changes occur
        break

    filtered_linknos = new_linknos  # Update for next iteration

print(filtered_linknos)

[710458938]
[720201520]
[720142375]
[730246043]
[740601736]
[760093117]
[730244763]
[760156342]
[730248604]
[730247324]
[710458938, 720201520, 720142375, 730140925, 730247324, 740601736, 760049883, 760156342, 770390434]


In [26]:
filtered_nexus_points = nexus_points[nexus_points["DSLINKNO"].isin(filtered_linknos)]
filtered_nexus_points

,Lat,Lon,DSLINKNO,DSStrahlerOrder,USLINKNOs,geometry
1142411,31.871444,-114.915778,710458938,9,"710375612,710456954",POINT (-12792365.866 3746447.517)
1159032,45.600960,-122.587674,720142375,9,"720120555,720140391",POINT (-13646397.420 5716631.804)
1166709,49.151637,-123.010758,720201520,8,"720157872,720215403",POINT (-13693494.896 6300630.266)
1174255,56.937547,-92.710477,730247324,9,"730153887,730251165",POINT (-10320483.103 7747364.438)
1175491,58.781107,-94.212990,730140925,8,"730085888,730139645",POINT (-10487742.118 8133225.404)
1191778,46.635516,-71.911809,740601736,9,"740381320,740599048",POINT (-8005185.951 5882782.555)
1241641,29.357098,-89.457095,760049883,9,"760034907,760071500",POINT (-9958318.224 3421175.583)
1262410,29.833654,-91.395373,760156342,8,"760091446,760141362",POINT (-10174086.400 3482185.455)
1283707,25.948556,-97.157889,770390434,8,"770308274,770358193",POINT (-10815566.718 2992710.723)


In [27]:
filtered_nexus_points.to_file("/Users/rachel1/Downloads/filtered_nexus_points_10000.gpkg", driver="GPKG")